In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**28
from matplotlib import animation
from IPython.display import HTML
from common import set_figure

#### Problem description

Here you will set up the problem for
$$ u_t + c u_x = 0$$
with periodic BC on the interval [0,1]

In [ ]:
c = 1.0
T = 1.0 / c # end time

#### Set up the grid

`hx` is the grid spacing in the `x`-direction

`x` are the grid coordinates

`xx` are really fine grid coordinates

In [ ]:
nx = 64
x = np.linspace(0, 1, nx, endpoint=False)
hx = x[1] - x[0]
xx = np.linspace(0, 1, 100, endpoint=False)
print(hx)

Now define an initial condition

In [ ]:
def square(x):
    u = np.zeros(x.shape)
    u[np.intersect1d(np.where(x>0.4), np.where(x<0.6))] = 1.0
    return u

f = square

In [ ]:
plt.plot(x, f(x), lw=3, clip_on=False)

#### Setting time step $h_t$

Now we need a time step.  Let
$$ h_t = h_x \frac{\lambda}{c}$$

So we need a parameter $\lambda$

<span style="color:red">What happens when $\lambda>1.0$?</span>

<span style="color:red">When the `method` changes to FTCS, what is the impact of $\lambda$?</span>

In [ ]:
lmbda = 0.05
ht = hx * lmbda / c
nt = int(T/ht)
print(f'     T = {T}')
print(f'tsteps = {nt}')
print(f'    hx = {hx}')
print(f'    ht = {ht}')
print(f'lambda = {lmbda}')

Now make an index list, called $J$, so that we can access $J+1$ and $J-1$ easily

In [ ]:
J = np.arange(0, nx)  # all vertices
Jm1 = np.roll(J, 1)
Jp1 = np.roll(J, -1)

# Run and Animate

In [ ]:
import time

method = 'ETBS'

u = f(x)

fig, ax = plt.subplots()

ax.set_title('u vs x')
ax.set_ylim(0, 1)
line1, = ax.plot(xx, f(xx), '-', color='tab:blue', lw=3, clip_on=False)
line2, = ax.plot(x, u, '-', color='tab:red', lw=3, clip_on=False)
    
def init():
    pass


def timestepper(n):
    
    if method == 'ETBS':
        u[J] = u[J] - lmbda * (u[J] - u[Jm1])
            
    if method == 'ETFS':
        u[J] = u[J] - lmbda * (u[Jp1] - u[J])
        
    if method == 'ETCS':
        u[J] = u[J] - lmbda * (1.0 / 2.0) * (u[Jp1] - u[Jm1])
    
    # exact solution
    uexact = f((xx - c * (n+1) * ht) % 1.0)
        
    line1.set_data(xx, uexact)
    line2.set_data(x, u)
    
    return line1, line2

ani = animation.FuncAnimation(fig, timestepper, 
                              frames=nt, interval=30,
                              init_func=init)
html = HTML(ani.to_jshtml())
plt.clf()
html

#### Check the error

In [ ]:
uexact = f((x - c * (nt+1) * ht) % 1.0)
error = u - uexact
l2err = np.sqrt(hx * np.sum(error**2))
print(l2err)